In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/barCharts.ipynb
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

Selectors :
   * xaxisCompletableScore : dropdown value indicating the number of bar per chart (Alls, 5, 25, 50, 10 first, 10 last)
   * orderCompletableScore : dropdown value indicating how to order xlabel : alphabetically or by y value (hight to low or low to hight)

In [ ]:
#constructing a dropdown to select how many data we want to see in visualisations for x axis
xaxisCompletableScore=widgets.Dropdown(options=[('All datas',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
          description='x axe',
          disabled=False)
#constructing a dropdown to order for alphabetic keys or for values
orderCompletableScore=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
          description='Order',
          disabled=False)

***vis_completables_scores* function**:
* Output : the widgets selector and the outputCompletableScore in a list to be display in a VBox
* outputCompletableScore : a bar chart showing the score achieved by players in the different completables

In [2]:
outputCompletableScore=widgets.Output()
def vis_completables_scores():
    def display_completable_score(change):
        with outputCompletableScore:
            clear_output(wait=False)
            getAllScorePerCompletable()
            clear_output(wait=True)
    xaxisCompletableScore.observe(display_completable_score,'value')
    orderCompletableScore.observe(display_completable_score,'value')
    display_completable_score(None)
    return [xaxisCompletableScore, orderCompletableScore, outputCompletableScoreUpdate, outputCompletableScore]

***getAllsScorePerCompletable* function**:

Output : Displays bar chart showing the score achieved by players in the different completables

In [ ]:
def getAllScorePerCompletable():
    #get all players selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    completableSelected=getSublistSelectedFromUnionListSelected(completable_score_list,
                                                                checkboxesCompletableSelected.get_selected_options())
    if len(completableSelected)>0 and len(playersSelected)>0:
        if xaxisCompletableScore.value in [0,-1,-2]:
            nbgraph=1
        else:
            div=len(completableSelected)/xaxisCompletableScore.value
            nbgraph=math.ceil(div)
        accordionCompletableScore=widgets.Accordion(children=[widgets.Output() for i in range(nbgraph)],
                                         selected_index=None)
        if nbgraph>1:
            for i in range(nbgraph):
                accordionCompletableScore.set_title(i, str(i+1)+"/"+str(nbgraph))
        display(accordionCompletableScore)
        def on_selected_index_completable_score_change(change):
            n=change['new']
            if n!=None:
                #close and clear previous fig of completable score
                for fig in figCompletableScore:
                    plt.close(fig=fig)
                figCompletableScore.clear()
                #get all players selected
                playersSelected=checkboxesPlayersSelected.get_selected_options()
                completableSelected=getSublistSelectedFromUnionListSelected(completable_score_list,
                                                                            checkboxesCompletableSelected.get_selected_options())
                players_scores = {}
                for player in playersSelected:
                    if players_info[player]["completables_scores"] != []:
                        for c in completableSelected:
                            if not c in players_scores.keys():
                                players_scores[c]={}
                            if c in players_info[player]["completables_scores"].keys():
                                try:
                                    players_scores[c][player] = int(players_info[player]["completables_scores"][c])
                                except:
                                    players_scores[c][player] = 0
                                    for value in players_info[player]["completables_scores"][c].values():
                                        players_scores[c][player]+=value
                            else:
                                players_scores[c][player] = 0
                #order alls values
                ordered_scores = orderDictByDictValues(players_scores,orderCompletableScore.value)
                #get positions of bar dict
                pos, bar_width = getPosDictMultiBarChart(ordered_scores,playersSelected)
                #separe in sub dict in function of xaxis dropout selected
                nbgraphshown, players_scores, N = organizeDictPerxAxisSelected(completableSelected,xaxisCompletableScore.value,players_scores)
                pos = organizeDictPosPerDataxaxis(xaxisCompletableScore.value,pos)
                with accordionCompletableScore.children[n]:
                    clear_output(wait=False)
                    #create new fig with custom name in the tab
                    name=filename+"_completable_scores"
                    if nbgraphshown>1:
                        name+="_"+str(n+1)+"_"+str(nbgraphshown)
                    fig = plt.figure(num=name, constrained_layout=True)
                    fig.canvas.header_visible = False
                    fig.canvas.layout.min_height = '400px'
                    figCompletableScore.append(fig)
                    # separe in sub dict in function of xaxis dropout selected
                    players_scores[n] = reorganiseDictBySubKeys(players_scores[n])
                    if len(completableSelected)>0 and len(playersSelected)>0:
                        # display multibar with vertical lines
                        constructMultiBarWithVerticalLines(players_scores,n,
                                                           playersSelected, completableSelected,
                                                           pos, bar_width, N)
                         #add title and labels to graph
                        plt.xlabel("Completable")
                        plt.ylabel("Score")
                        plt.xticks(rotation=90)
                        plt.title("Scores of players per completable")
                        plt.show()
        accordionCompletableScore.observe(on_selected_index_completable_score_change, 'selected_index')
        accordionCompletableScore.selected_index=0
    else:
        #create new fig with custom name in the tab
        name=filename+"_completable_scores"
        fig = plt.figure(num=name, constrained_layout=True)
        fig.canvas.header_visible = False
        fig.canvas.layout.min_height = '400px'
        figCompletableScore.append(fig)
        #display no data to fill vis
        noDataToFillVis(100)
        #add title and labels to graph
        plt.xlabel("Completable")
        plt.ylabel("Score")
        plt.xticks(rotation=90)
        plt.title("Scores of players per completable")
        plt.show()
    with outputCompletableScoreUpdate:
        clear_output(wait=False)